# Opérateur d'intérêt : Förstner

L'opérateur de **Förstner** identifie également les points d’intérêt à l'aide de la matrice <code>M</code>.  
Or, l’algorithme de Förstner est utilisé pour obtenir une solution approximative avec une précision sous-pixel. Il résout le point le plus proche de toutes les lignes tangentes du coin dans une fenêtre donnée. L'algorithme repose sur le fait que pour un coin idéal, les lignes tangentes se croisent en un seul point.

## Algorithme

1. Conversion de l'image couleur en niveaux de gris
2. Calcul des images du gradient <code>Ix</code> et <code>Iy</code>
3. Calcul la matrice de variances covariances du gradient <code>M</code>
4. Calcul des éléments de l'ellipse d'erreur pour chaque pixel <code>w</code> et <code>q</code>
4. Calcul de la réponse de Förstner
5. Identification des coins comme points extrêmes qui prennent les plus grandes valeurs des points candidats

#### Étape 1 : Convertir l'image en niveaux de gris

In [ ]:
from skimage.io import imread
from skimage.color import rgb2gray
import os

os.chdir("C:/Users/Anass/Desktop/Corner Detector/data")
img = imread("Aerial Photo.jpg")
imgGray = rgb2gray(img)
imgGray

#### Étape 2 : Calcul des images du gradient <code>Ix</code> et <code>Iy</code>

Le gradient peut se calculer toujours avec l'opérateur de Sobel:

In [ ]:
from scipy import signal as sig
import numpy as np

def gradient_x(imgGray):
    kernel_x = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]])
    return sig.convolve2d(imgGray, kernel_x, mode='same')

def gradient_y(imgGray):
    kernel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    return sig.convolve2d(imgGray, kernel_y, mode='same')

I_x = gradient_x(imgGray)
I_y = gradient_y(imgGray)

#### Étape 3 : Calcul la matrice de variances covariances du gradient <code>M</code>

In [ ]:
from scipy import ndimage as ndi

# scipy.ndimage.gaussian_filter(input, sigma)
Ixx = ndi.gaussian_filter(I_x**2, sigma=1)
Ixy = ndi.gaussian_filter(I_y*I_x, sigma=1)
Iyy = ndi.gaussian_filter(I_y**2, sigma=1)

#### Étape 4 : Calcul des éléments des ellipses d'erreur <code>w</code> et <code>q</code>

Förstner prend en compte les deux valeurs propres <code>λ1</code> et <code>λ2</code> de l'inverse de la matrice comme valeur d'intérêt. Elles définissent les axes d'une ellipse d'erreur. Par le calcul de leur taille :

![img](https://i.ibb.co/kmq6DQ0/Taille-ellipse.png)

Et le facteur de forme relatif à la rondeur:

![img](https://i.ibb.co/cJfWYv2/Roudness.png)

Les propriétés suivantes peuvent être déduites :
* Les petites ellipses circulaires définissent un point d’intérêt
* Les ellipses d'erreur allongées suggèrent un bord droit
* Les grandes ellipses marquent une zone homogène

Un point d'intérêt est présent si les valeurs seuils données T_w et T_q sont dépassées. Les paramètres 	appropriés pour cela se situent dans l'intervalle :

![img](https://i.ibb.co/FX4gb1W/Seuils.png)

In [ ]:
n,m = img.shape[0],img.shape[1]

# Seuil Tq
Tq = 0.5

# Initiation de la matrice de précision w
w = np.zeros([n,m])

for i in range(1,n-2):
    for j in range(1,m-2):
        detM = Ixx[i,j] * Iyy[i,j] - Ixy[i,j] ** 2
        traceM = Ixx[i,j] + Iyy[i,j]
        q = 4 * detM / (traceM ** 2)
        if q >= Tq:
            w[i,j] = detM / traceM

In [ ]:
# Vérification de la deuxième condition
Wm = 0
for i in range(0,n-1):
    for j in range(0,m-1):
        Wm += w[i,j]
        
Wm = Wm / (n * m)

# Seuil Tw
Tw = 15 * Wm

#### Étape 5 : Calcul de la réponse de Förstner

In [ ]:
# Initiation de la réponse de Forstner
forstner_response = np.zeros([n,m])

for i in range(0,n):
    for j in range(0,m):
        if w[i,j] >= Tw:
            forstner_response[i,j] = 255
        else:
            forstner_response[i,j] = 0

#### Étape 6 : Identification des coins comme points extrêmes qui prennent les plus grandes valeurs des points candidats

In [ ]:
import matplotlib.pyplot as plt
img_copy_for_corners = np.copy(img)

for i in range(0,n):
    for j in range(0,m):
        if forstner_response[i,j] == 255:
            # c'est un coin
            img_copy_for_corners[i,j] = [255,0,0]

fig, ax = plt.subplots(figsize=(17,17))
ax.imshow(img_copy_for_corners, interpolation='nearest', cmap=plt.cm.gray)

**Remarque** : l'algorithme identifie les régions d'intérêts. Pour trouver les coins, nous pouvons utiliser un algorithme de recherche de pic

In [ ]:
import matplotlib.pyplot as plt
from skimage.feature import corner_peaks

corners = corner_peaks(forstner_response)
fig, ax = plt.subplots(figsize=(17,17))
ax.imshow(img, interpolation='nearest', cmap=plt.cm.gray)
ax.plot(corners[:, 1], corners[:, 0], '.r', markersize=3)